In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('resources/database.sqlite') 

In [3]:
df2 = pd.read_sql_query("SELECT * FROM player_attributes INNER JOIN Player ON Player.id = player_attributes.id", conn)
df2.shape

(11060, 49)

In [4]:
df1 = pd.read_csv('resources/final_data.csv')
df1.rename(columns={'name': 'player_name'}, inplace=True)

In [5]:
merged_df = pd.merge(df1, df2, on='player_name')
merged_df = merged_df.dropna()  # Drop rows with missing values
merged_df = merged_df.drop_duplicates(subset='player_name')  

merged_df.shape

(1220, 70)

In [6]:
merged_df['goal_per_appearance'] = merged_df['goals'] / merged_df['appearance']
merged_df['assist_per_appearance'] = merged_df['assists'] / merged_df['appearance']

In [7]:
merged_df.head()


,player,team,player_name,position,height_x,age,appearance,goals,assists,yellow cards,...,gk_positioning,gk_reflexes,id,player_api_id,player_fifa_api_id,birthday,height_y,weight,goal_per_appearance,assist_per_appearance
0,/jack-butland/profil/spieler/128899,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.000000,0.000000,0.069018,...,69.0,68.0,4502,288880,203042,1993-03-10 00:00:00,195.58,209,0.000000,0.000000
1,/tom-heaton/profil/spieler/34130,Manchester United,Tom Heaton,Goalkeeper,188.0,37.0,4,0.000000,0.000000,0.000000,...,75.0,78.0,10377,24155,163264,1986-04-15 00:00:00,187.96,190,0.000000,0.000000
2,/phil-jones/profil/spieler/117996,Manchester United,Phil Jones,Defender Centre-Back,185.0,31.0,8,0.000000,0.000000,0.216346,...,68.0,72.0,8589,186137,194957,1992-02-21 00:00:00,185.42,157,0.000000,0.000000
3,/luke-shaw/profil/spieler/183288,Manchester United,Luke Shaw,Defender Left-Back,178.0,27.0,74,0.015374,0.169115,0.353604,...,13.0,8.0,6432,362694,205988,1995-07-12 00:00:00,185.42,165,0.000208,0.002285
4,/casemiro/profil/spieler/16306,Manchester United,Casemiro,midfield-DefensiveMidfield,185.0,31.0,101,0.090817,0.124874,0.329213,...,70.0,75.0,1596,208494,200145,1992-02-23 00:00:00,182.88,176,0.000899,0.001236


In [8]:
merged_df.to_csv("resources/test.csv", index=False)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = merged_df.select_dtypes(include=[np.number])


X = pd.get_dummies(merged_df, drop_first=True)
X = X.dropna()

y = merged_df['goals']
y = y[X.index] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.0008381504680398298


In [10]:
print(y.describe())

count    1211.000000
mean        0.146785
std         0.191631
min         0.000000
25%         0.000000
50%         0.070625
75%         0.219275
max         2.000000
Name: goals, dtype: float64


In [11]:
rmse = np.sqrt(0.0008381504680398298)
print("RMSE:", rmse)

RMSE: 0.028950828451701167
